# Pip install all needed packages

In [1]:
!pip install gym
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


# Imports

In [16]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# Global variables

In [25]:
depthOfCode = 10
rows = 3
cols = 3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Functions

In [26]:
# swap is given actions which is a tuple of actions or a action, where every action is a tuple with the values
# of two qubits (x, y) whos values should be swaped. x and y are ints between 0 and 8 corresponding to 
# the following qubit notation:
#         [[0, 1, 2],
#          [3, 4, 5],
#          [6, 7, 8]]

# ex. of a tuple of actions: ((0, 3), (4, 5), (7, 8))

# ex. of a single action: (0, 1)

# in the case of a single action we make a list out of it so it's iterable to minimize code
def swap(state, actions):
    if type(actions[0]) != tuple:
        actions = [actions]
    for action in actions:
        pos0, pos1 = action
        
        col0 = pos0%cols
        row0 = int((pos0-col0)/cols)  
        col1 = pos1%cols
        row1 = int((pos1-col1)/cols)
        
        for i in range(len(state)):
            state[i][row0][col0], state[i][row1][col1] = state[i][row1][col1], state[i][row0][col0]


In [27]:
# getNeighbors returns a list of the qubit notations of all neighbors to a specific qubit. 
# I.e. qubits above, below, right and left of the specific qubit.

def getNeighbors(state, row_number, column_number):
    a = [[state[i][j] if  i >= 0 and i < len(state) and j >= 0 and j < len(state[0]) else -1
                    for j in range(column_number-1, column_number+2)]
                        for i in range(row_number-1, row_number+2)]
    return [a[0][1], a[1][0], a[1][2], a[2][1]]

In [28]:
#                         [[1,0,0], [[1,0,0],   [[1,0,0],         [[1,0,0],  
# Takes in a state like [  [1,0,2],  [1,0,2], ,  [1,0,2], , ... ,  [1,0,2], ] and checks if all the pairs of 
#                          [2,0,0]]  [2,0,0]]    [2,0,0]]          [2,0,0]] 
# numbers in the first slice are neighbors and if so returns True else returns False

def isExecutableState(state):
    for row in range(len(state[0])):
        for col in range(len(state[0][0])):
            if state[0][row][col] > 0:
                if not state[0][row][col] in getNeighbors(state[0], row, col):
                    return False

    return True

In [29]:
# We use this once to get all the different swap combinations. I.e. all acceptable combinations of one to four
# swaps. This are the different actions we cound make in one timestep.

def getPossibleActions(maxSwapsPerTimeStep=math.floor(rows*cols/2)):
    state = np.arange(rows*cols).reshape((rows,cols))
    
    possibleActions = getPossibleActionsSub(state, [], maxSwapsPerTimeStep)
    
    possibleActions = set(map(lambda x: tuple(sorted(x)), possibleActions ))
    
    possibleActions = list(possibleActions)
    possibleActions.append((0, 0))
    
    return possibleActions
    
def getPossibleActionsSub(state, used, maxSwapsPerTimeStep):
    if maxSwapsPerTimeStep == 0:
        return np.asarray([])
    
    possibleActions = []
    
    for i in range(len(state)):
        for j in range(len(state[0])):
            
            usedtmp = used.copy()
            
            if not state[i][j] in usedtmp:
                neighbors = getNeighbors(state, i, j)
                for neighbor in neighbors:
                    if neighbor >= 0 and not (neighbor, state[i][j]) in possibleActions and not neighbor in usedtmp:
                        possibleActions.append((state[i][j], neighbor))
                        usedtmp.append(state[i][j])
                        usedtmp.append(neighbor)
 
                        for action in getPossibleActionsSub(state, usedtmp, maxSwapsPerTimeStep-1):
                            if type(action) == tuple:
                                possibleActions.append([(state[i][j], neighbor), action])
                            elif type(action) == list:
                                action.append((state[i][j], neighbor))
                                possibleActions.append(action)
                                
        
    return possibleActions

In [30]:
# Creates a shuffled Matrix simulatinga slice of quantum code with one to max amount 
# of operations per timestep

# Ex1. [[0, 1, 0],
#       [1, 2, 2],
#       [3, 0, 3]]

# Ex2. [[2, 1],
#       [2, 1]]

def makeStateSlice():
    random = np.random.choice([x for x in range(2, rows*cols+2) if x % 2])
    stateSlice = np.ceil(np.arange(1, random)/2)
    stateSlice = np.append(stateSlice, np.zeros(rows*cols-random+1))
    np.random.shuffle(stateSlice)
    return stateSlice.reshape((rows,cols))

In [31]:
# Makes a state out of depthOfCode amount of slices
def makeState():
    state = np.zeros((depthOfCode,rows,cols))
    for i in range(len(state)):
        state[i] = makeStateSlice()
    return state

# Tests some of the functions

In [32]:
state = np.zeros((depthOfCode,rows,cols))
for i in range(len(state)):
    state[i] = np.arange(rows*cols).reshape((rows,cols))

#print(state)

possibleActions = getPossibleActions()
print(len(possibleActions))
#print(possibleActions)

a = possibleActions[0]

print(a)

statetmp = np.arange(rows*cols).reshape((rows,cols))

swap(state, a)

print(state[0])

#for i in range(len(possibleActions)):
#output = set(map(lambda x: tuple(sorted(x)),possibleActions))

#print(len(output))

131
((2, 5), (7, 8))
[[0. 1. 5.]
 [3. 4. 2.]
 [6. 8. 7.]]


Test function makeState

In [33]:
print(makeState())

[[[4. 2. 1.]
  [1. 0. 3.]
  [3. 2. 4.]]

 [[3. 4. 1.]
  [2. 0. 1.]
  [2. 4. 3.]]

 [[1. 0. 0.]
  [3. 0. 2.]
  [2. 1. 3.]]

 [[0. 1. 0.]
  [0. 0. 0.]
  [0. 1. 0.]]

 [[0. 1. 1.]
  [3. 2. 4.]
  [3. 2. 4.]]

 [[1. 0. 2.]
  [2. 3. 0.]
  [1. 0. 3.]]

 [[0. 0. 0.]
  [0. 1. 0.]
  [0. 1. 0.]]

 [[2. 1. 2.]
  [0. 3. 1.]
  [0. 0. 3.]]

 [[1. 3. 2.]
  [2. 4. 4.]
  [1. 3. 0.]]

 [[1. 3. 0.]
  [2. 0. 1.]
  [2. 0. 3.]]]


# Enviotment definition and sub functions

In [34]:
#Our enviorment
class Kvant(Env):
    def __init__(self):
        #array of possible actions
        self.possibleActions = getPossibleActions()
        #self.possibleActions = getPossibleActions(1) #this for only 1 swap at a time
        
        #Number of actions we can take
        self.action_space = Discrete(len(self.possibleActions))
        
        #
        self.observation_space = Box(low=np.zeros((depthOfCode,rows,cols), dtype=int), high=np.zeros((depthOfCode,rows,cols), dtype=int)+4)
        
        #The start state
        self.state = makeState()
        
        #max amount of swaps between layers
        #self.maxTimeSteps = 5
        
        #max amount of layers per episode
        self.maxLayers = depthOfCode
        
    def step(self, action):
        
        actions = self.possibleActions[action]

        swap(self.state, actions)
         
        if isExecutableState(self.state):
            # reset the maxSwaps to 5
            self.maxTimeSteps = 5
            
            # remove the exicutable slice and add a new random slice at the tail
            self.state = np.roll(self.state, -1, axis = 0)
            self.state[depthOfCode - 1] = makeStateSlice()
            
            self.maxLayers -= 1
            
            # we are not done except if this was the last layer we can work on this episode
            if self.maxLayers <= 0:
                done = True
            else:
                done = False
            
        else:
            done = False
        
        # Rewards 
        if actions == (0,0):
            reward = 0
        
        else:
            reward = -1
        
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        pass
    
    def reset(self):
        self.state = makeState()
        #self.maxTimeSteps = 5
        self.maxLayers = 10
        return self.state

# Testing the enviorment

In [35]:
env = Kvant()

## test just random actions

In [38]:
episodes = 100
scores = []
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    scores.append(score)

In [39]:
for i in range(min(scores), max(scores)+1):
    if scores.count(i) != 0:
        print('Number of', i, 'is', scores.count(i))

Number of -551 is 1
Number of -550 is 1
Number of -545 is 1
Number of -523 is 1
Number of -515 is 1
Number of -514 is 1
Number of -512 is 1
Number of -473 is 1
Number of -466 is 1
Number of -460 is 1
Number of -456 is 1
Number of -449 is 1
Number of -438 is 1
Number of -430 is 2
Number of -414 is 1
Number of -402 is 1
Number of -388 is 1
Number of -383 is 1
Number of -381 is 1
Number of -375 is 1
Number of -374 is 1
Number of -365 is 1
Number of -349 is 1
Number of -342 is 1
Number of -340 is 1
Number of -337 is 1
Number of -334 is 1
Number of -329 is 1
Number of -326 is 1
Number of -323 is 1
Number of -321 is 1
Number of -317 is 1
Number of -311 is 1
Number of -309 is 2
Number of -306 is 3
Number of -302 is 2
Number of -297 is 1
Number of -295 is 1
Number of -289 is 1
Number of -285 is 1
Number of -275 is 1
Number of -271 is 2
Number of -270 is 1
Number of -269 is 1
Number of -265 is 1
Number of -260 is 1
Number of -257 is 2
Number of -247 is 1
Number of -244 is 1
Number of -243 is 1


# Replay memory

# Q network

# Training

# Training loop